## Final data preprocessing - Bias Data

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import json

In [ ]:
# set the max columns to none
pd.set_option('display.max_columns', None)

In [ ]:
## Load datasets
df_training = pd.read_excel("\data\processed_bias_data\20240131_training_processed.xlsx")
df_pilot = pd.read_excel("\data\processed_bias_data\20240131_pilot_processed.xlsx")
df_prepilot = pd.read_excel("\data\processed_bias_data\20240131_prepilot_processed.xlsx")

In [ ]:
list_dfs = [df_training, df_prepilot, df_pilot]

In [ ]:
# for df in list_dfs:
#     display(df.columns)
#     # display(df.head())

In [ ]:
# These are the columns of the dataset used in the bias analysis of the pilot, plus 'is_parttime_parent' and 'is_fulltime_parent'
# This can be found in C:\Users\berker007\OneDrive - Gemeente Amsterdam\Team AA\Projecten\Slimme Check\Bias Analyse\Code & Documentatie Bias Analyse Pilot\Bias_analyse_code_Loek_v2\preprocessing.ipynb

bias_columns = ['Index', 
                'SlimmeCheck.PREDICTION', 
                'Slimme Check', 'IO NIO',
       'Sherlock.Resultaat onderzoek', 
       'Leeftijd', 
       'Bias.Nationaliteit',
       'Bias.Geslacht', 
       'SlimmeCheck.SCORE',
       'SlimmeCheck.VALUE_AFSPRAKEN_NO_CONTACT_COUNT_LAST_YEAR',
       'SlimmeCheck.VALUE_APPLIED_FOR_SAME_PRODUCT_LAST_YEAR',
       'SlimmeCheck.VALUE_TOTAL_VERMOGEN',
       'SlimmeCheck.VALUE_DAYS_SINCE_LAST_DIENST_END',
       'SlimmeCheck.VALUE_DAYS_SINCE_LAST_RELOCATION',
       'SlimmeCheck.VALUE_AT_LEAST_ONE_ADDRESS_IN_AMSTERDAM',
       'SlimmeCheck.VALUE_AFSPRAKEN_NO_SHOW_COUNT_LAST_YEAR',
       'SlimmeCheck.VALUE_RECEIVED_SAME_PRODUCT_LAST_YEAR',
       'SlimmeCheck.VALUE_DEELNAMES_STARTED_PERCENTAGE_LAST_YEAR',
       'SlimmeCheck.VALUE_ACTIVE_ADDRESS_COUNT',
       'SlimmeCheck.VALUE_HAS_PARTNER', 
       'SlimmeCheck.VALUE_HAS_MEDEBEWONER',
       'SlimmeCheck.VALUE_AVG_PERCENTAGE_MAATREGEL',
       'SlimmeCheck.VALUE_SUM_INKOMEN_BRUTO_VALUE',
       'SlimmeCheck.VALUE_SUM_INKOMEN_BRUTO_WAS_MEAN_IMPUTED', 
       'Leeftijd<30',
       'Leeftijd<40', 
       'Leeftijd<50', 
       'IsNederlands', 
       'IsWesters',
       'Days_since_last_dienst_end_year', 
       'Days_since_last_dienst_end_months',
       'Days_since_last_relocation', 
       'has_single_address',
       'afspraken_no_show_count', 
       'afspraken_no_contact_count',
       'resultaat_handhaving',
       'is_parttime_parent', 
       'is_fulltime_parent']

### df_training & df_prepilot columns

In [ ]:
import datetime

In [ ]:
df_training['onderzoekswaardig'].value_counts()

In [ ]:
# Mutations to df_training to align with df_prepilot
df_training = df_training.rename(columns = {'onderzoekswaardig':'Label',
                                            'GESLACHT':'geslacht'})
df_training = df_training.drop(columns = ['DTGEBOORTE', 'is_onderzoek_hh'], errors='ignore')

In [ ]:
# Mutations to df_prepilot to align with df_training
df_prepilot.loc[df_prepilot['geboortejaar'].isna(), 'geboortejaar'] = df_prepilot['geboortejaar'].mean()
df_prepilot['geboortejaar'] = df_prepilot['geboortejaar'].astype(int)
df_prepilot['dtaanvraag'] = pd.to_datetime(df_prepilot['dtaanvraag'])
df_prepilot = df_prepilot.drop(columns = ['DTGEBOORTE', 'geboortejaar'])

In [ ]:
# # We want everyones age on the dtaanvraag
# # First give everyone a geboortedatum of 1st of July for their geboortejaar
# # We don't know actual birthdays, but for our goal this is fine.
# df_prepilot['geboortedatum'] = df_prepilot['geboortejaar'].apply(lambda year: pd.to_datetime(f'{year}-07-01'))
# # Then we calculate age and store the result in a new column 'Leeftijd'
# df_prepilot['Leeftijd'] = np.floor((df_prepilot['dtaanvraag'] - df_prepilot['geboortedatum']) / np.timedelta64(1, 'Y'))
# # Then we can drop
# df_prepilot = df_prepilot.drop(columns = ['geboortedatum', 'geboortejaar'])


In [ ]:
# set(df_training.columns).intersection(set(df_prepilot.columns))

In [ ]:
set(df_training.columns).difference(set(df_prepilot.columns))

In [ ]:
set(df_prepilot.columns).difference(set(df_training.columns))

So now we have almost the same columns for df_prepilot and df_training.
Now filter out the ones that we don't need

In [ ]:
drop_columns = ['Unnamed: 0',
                'dtaanvraag',
                'DIENSTNR',
                'NATIONALITEIT1',
                'model_prob',
                'onderzoekswaardig']

def drop_columns(df:pd.DataFrame, drop_columns:list = drop_columns) -> pd.DataFrame:
    df = df.drop(columns = drop_columns, errors='ignore')
    return df

In [ ]:
df_training = drop_columns(df_training)
df_prepilot = drop_columns(df_prepilot)

### Pilot & the rest

In [ ]:
df_pilot[['onderzoekswaardig', 'Label']]

In [ ]:
df_pilot[['SlimmeCheck.SCORE', 'model_after_reweighing_score']]

### Create columns for most important features df_pilot

In [ ]:
contrib_columns = ['SlimmeCheck.CONTRIB_DEELNAMES_STARTED_PERCENTAGE_LAST_YEAR',
       'SlimmeCheck.CONTRIB_AFSPRAKEN_NO_CONTACT_COUNT_LAST_YEAR',
       'SlimmeCheck.CONTRIB_APPLIED_FOR_SAME_PRODUCT_LAST_YEAR',
       'SlimmeCheck.CONTRIB_RECEIVED_SAME_PRODUCT_LAST_YEAR',
       'SlimmeCheck.CONTRIB_TOTAL_VERMOGEN',
       'SlimmeCheck.CONTRIB_DAYS_SINCE_LAST_DIENST_END',
       'SlimmeCheck.CONTRIB_AT_LEAST_ONE_ADDRESS_IN_AMSTERDAM',
       'SlimmeCheck.CONTRIB_AFSPRAKEN_NO_SHOW_COUNT_LAST_YEAR',
       'SlimmeCheck.CONTRIB_DAYS_SINCE_LAST_RELOCATION',
       'SlimmeCheck.CONTRIB_ACTIVE_ADDRESS_COUNT',
       'SlimmeCheck.CONTRIB_HAS_PARTNER',
       'SlimmeCheck.CONTRIB_HAS_MEDEBEWONER',
       'SlimmeCheck.CONTRIB_AVG_PERCENTAGE_MAATREGEL',
       'SlimmeCheck.CONTRIB_SUM_INKOMEN_BRUTO_VALUE',
       'SlimmeCheck.CONTRIB_SUM_INKOMEN_BRUTO_WAS_MEAN_IMPUTED']

In [ ]:
# We want the highest contributing features
def n_highest(row, n):
    row = row.abs()
    sorted_row = sorted(row, reverse=True)
    n_highest_value = sorted_row[n]
    n_highest_column = row.index[row == n_highest_value][0]
    return n_highest_column

In [ ]:
# # We want the highest contributing features
# def n_highest(row, n):
#     sorted_row = sorted(row, reverse=True)
#     n_highest_value = sorted_row[n]
#     n_highest_column = row.index[row == n_highest_value][0]  
#     return n_highest_column

In [ ]:
sorted(df_pilot[contrib_columns].iloc[0,:].abs(),reverse=True)

In [ ]:
df_pilot['SlimmeCheck.CONTRIB_AFSPRAKEN_NO_SHOW_COUNT_LAST_YEAR'].value_counts()

In [ ]:
df_prepilot['Belangrijkste feature 1'].value_counts()

In [ ]:
# def check_pos_neg(value, df = df_pilot):
#     if value > 0:
#         return ' (+)'
#     elif value < 0:
#         return ' (-)'
#     else:
#         return ' (0)'

# While we know the highest contributors, we still have to know if it was a positive or negative contribution to adhere to the style used in the pilot output    
def check_pos_neg(value, df=df_pilot):
    try:
        col_value = df[value]
        positive_mask = col_value > 0
        negative_mask = col_value < 0
        zero_mask = col_value == 0
        
        result = pd.Series(index=col_value.index, dtype=str)
        result[positive_mask] = ' (+)'
        result[negative_mask] = ' (-)'
        result[zero_mask] = ' (0)'
        
        return result.fillna(' (Column Not Found)')
    except KeyError:
        return ' (Column Not Found)'

In [ ]:
belangrijkste_feature_columnnames = ['Belangrijkste feature 1', 'Belangrijkste feature 2', 'Belangrijkste feature 3']

for i, imp_feature in enumerate(belangrijkste_feature_columnnames):
    df_pilot[imp_feature] = df_pilot[contrib_columns].apply(lambda row: n_highest(row, i), axis=1)
    df_pilot[f"{imp_feature}_pos_neg"] = df_pilot[imp_feature].apply(check_pos_neg)[0]
    # df_pilot[imp_feature] = f"{df_pilot[imp_feature]}{(df_pilot[imp_feature].apply(lambda row: check_pos_neg(row, imp_feature)))}"
    
    df_pilot[imp_feature] = df_pilot[imp_feature].apply(lambda x: x.split('SlimmeCheck.CONTRIB_')[1].lower())

In [ ]:
# We need to rewrite the features
feature_name_mapping = {
    "deelnames_started_percentage_last_year": "Percentage deelnames gestart",
    "at_least_one_address_in_amsterdam": "Adres in Amsterdam",
    "active_address_count": "Aantal adressen",
    "days_since_last_relocation": "Dagen sinds verhuizing",
    "days_since_last_dienst_end": "Dagen sinds einde dienst",
    "has_medebewoner": "Medebewoner",
    "avg_percentage_maatregel": "Gemiddelde percentage maatregel",
    "total_vermogen": "Totaal vermogen",
    "afspraken_no_show_count_last_year": "Aantal afspraken no show",
    "has_partner": "Partner",
    "sum_inkomen_bruto_was_mean_imputed": "Inkomen onbekend",
    "applied_for_same_product_last_year": "Eerder Levensonderhoud aangevraagd",
    "received_same_product_last_year": "Eerder Levensonderhoud ontvangen",
    "afspraken_no_contact_count_last_year": "Aantal afspraken geen contact",
    "sum_inkomen_bruto_value": "Totaal bruto inkomen",
}

for imp_feature in belangrijkste_feature_columnnames:
    df_pilot[imp_feature] = df_pilot[imp_feature].replace(feature_name_mapping)
    df_pilot[imp_feature] = df_pilot[imp_feature] + df_pilot[f"{imp_feature}_pos_neg"]

In [ ]:
df_pilot['Belangrijkste feature 1'].value_counts()

In [ ]:
df_prepilot['Belangrijkste feature 1'].value_counts()

### Filter the columns of df_pilot

In [ ]:
columns_pilot_for_bias_analysis = ['Leeftijd', 'Bias.Nationaliteit', 'Bias.Geslacht', 'applied_for_same_product_last_year', 'total_vermogen',
       'days_since_last_dienst_end', 'days_since_last_relocation',
       'at_least_one_address_in_amsterdam',
       'afspraken_no_show_count_last_year', 'received_same_product_last_year',
       'deelnames_started_percentage_last_year', 'active_address_count',
       'model_before_reweighing_score',
       'model_before_reweighing_prediction',
       'model_after_reweighing_score',
       'model_after_reweighing_prediction',
       'onderzoekswaardig',
       'Belangrijkste feature 1', 'Belangrijkste feature 2', 'Belangrijkste feature 3']

In [ ]:
# Here we filter for the columns that have the same info we have in df_prepilot and df_training
# This is almost all data we need for the bias analysis
df_pilot = df_pilot[columns_pilot_for_bias_analysis]

In [ ]:
df_pilot['Bias.Geslacht'].value_counts()

In [ ]:
# We don't have the data for these two columns for the pilot, so we add them as NaN
df_pilot['is_parttime_parent'] = np.nan
df_pilot['is_fulltime_parent'] = np.nan


In [ ]:
df_pilot = df_pilot.rename(columns = { 'Bias.Nationaliteit': 'NATIONALITEIT1_OMSCHRIJVING', 
 'Bias.Geslacht': 'geslacht', 
 'onderzoekswaardig': 'Label'})

In [ ]:
# We want to concat for easier manipulation but we want to keep info about the origin of the data
df_pilot['dataset'] = 'Pilot'
df_prepilot['dataset'] = 'Prepilot'
# df_training['dataset'] = 'Training'

In [ ]:
df_concat = pd.concat([df_pilot, df_prepilot, df_training])

## Feature engineering
- Now we want the features that are necessary for the bias analysis.
- We should have all the required information for this.
- This section is partly based on the work of Tess for the bias analysis of the pilot

### Leeftijd

In [ ]:
# There are some NAN, we fill those with median as is described in the technical documentation
df_concat.loc[df_concat['Leeftijd'].isna(), 'Leeftijd'] = df_concat['Leeftijd'].median()

In [ ]:
df_concat['Leeftijd'] = df_concat['Leeftijd'].astype(int)

In [ ]:
# Generate generic function to map age based on threshold
def map_age(age, threshold_age):
    if age < threshold_age:
        return 1
    elif age >= threshold_age:
        return 0
    else:
        return 2

In [ ]:
# Create three new columns in the dataset
    # 1. Leeftijd<30, where 1 = 0-29 and 0 = 30+
    # 2. Leeftijd<40, where 1 = 0-39 and 0 = 40+
    # 3. Leeftijd<50, where 1 = 0-49 and 0 = 50+
threshold_age = [30,40,50]

for age in threshold_age:
    df_concat[f'Leeftijd<{age}'] = df_concat['Leeftijd'].apply(map_age, args=(age,))
    print(df_concat[f'Leeftijd<{age}'].value_counts())

### Nationaliteit


In [ ]:
# Check unique values in feature Bias.Nationaliteit
all_nationalities = df_concat['NATIONALITEIT1_OMSCHRIJVING'].unique()
len(all_nationalities)

In [ ]:
# Creating a new column 'IsNederlands' where: 0 = Niet-Nederlands, 1 = Nederlands, 2 = Onbekende Nationaliteit
# Please note: It is decided to include the values: 'Onbekend' as unkown nationality (same as missing values)
def is_nederlands(nationality):
    if nationality == 'Nederlandse':
        return 1
    elif pd.isna(nationality):
        return 2
    elif nationality in ["Onbekend"]:
        return 2
    else:
        return 0
    
df_concat['IsNederlands'] = df_concat['NATIONALITEIT1_OMSCHRIJVING'].apply(is_nederlands)
df_concat['IsNederlands'].value_counts()

In [ ]:
# Load json file where Western and Non-Western nationalities are defined
# These files are the same as used during the pre-pilot bias analysis
# This is based on Annex 1 in the pre-pilot Bias analysis documentation: https://algoritmeregister.amsterdam.nl/ai-system/onderzoekswaardigheid-slimme-check-levensonderhoud/1086/
json_file_path = '\data\west-nonwest-nationalities.json'

with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
# Check if all nationalities in the dataset are categorized in one of the lists (western or non-western)
all_nationalities_set =set(all_nationalities)
western_set = set(data['west'])
non_western_set = set(data['nonwest'])

if all_nationalities_set.issubset(western_set.union(non_western_set)):
    print('All nationalities in the dataset are categorized')
else: 
    uncategorized = all_nationalities_set - western_set.union(non_western_set)
    print("The following nationalities are not categorized:")
    for nationality in uncategorized:
        print (nationality)


In [ ]:
# Create new column 'IsWesters', where 0 = Niet westerse nationaliteit, 1 = Westerse nationaliteit, 2 = Onbekende Nationaliteit
def is_westers(nationality):
    if nationality in western_set:
        return 1
    elif nationality in non_western_set:
        return 0
    else:
        return 2
    
df_concat['IsWesters'] = df_concat['NATIONALITEIT1_OMSCHRIJVING'].apply(is_westers)
df_concat['IsWesters'].value_counts()

### Geslacht

In [ ]:
df_concat['geslacht'].value_counts()

In [ ]:
# We have both M/V/O and 1/2/0 as sex categorization, make uniform
dict_map_sex_code = {1 : 'M',
                     2 : 'V',
                     0 : 'O'}

df_concat['geslacht'] = df_concat['geslacht'].replace(dict_map_sex_code)

# There are some nan, let's call it Onbekend
df_concat['geslacht'] = df_concat['geslacht'].fillna('Onbekend')

# Check result 
expected_values =  {'M', 'V', 'O', 'Onbekend'}
assert set(df_concat['geslacht'].unique()) == expected_values, "geslacht not equal to expected values. Please, make sure to clean the dataset"

df_concat['geslacht'].value_counts()


In [ ]:
df_concat.loc[df_concat['dataset'] == 'Pilot', 'geslacht'].value_counts()

### Prepare Feature: days_since_last_dienst_end & days_since_last_relocation

In [ ]:
# Generate generic function to map days_since_last_dienst_end and days_since_last_relocation based on threshold
def map_days(days, threshold_days):
    if days > threshold_days:
        return 1
    elif days <= threshold_days:
        return 0
    else:
        return 2

In [ ]:
# Apply function map_days to create three new columns:
    # 1. Days_since_last_dienst_end_year, where 1 = meer dan een jaar (>365), 0 = minder dan een jaar (<=365)
    # 2. Days_since_last_dienst_end_months, where 1 = meer dan twee maanden (>60), 0 = minder dan 2 maanden (<=60)
    # 3. Days_since_last_relocation, where 1 = meer dan een jaar (>365), 0 = minder dan een jaar (<=365)

def calculate_days_since_last(df, input_col, output_col, threshold):
    df[output_col] = df[input_col].apply(map_days, args=(threshold,))
    return df[output_col].value_counts()

dict_days_since_last = [{'input_col': 'days_since_last_dienst_end', 'output_col': 'Days_since_last_dienst_end_year', 'threshold':365}, 
                        {'input_col': 'days_since_last_dienst_end', 'output_col':'Days_since_last_dienst_end_months', 'threshold':60}, 
                        {'input_col': 'days_since_last_relocation', 'output_col':'Days_since_last_relocation', 'threshold':365}]

for dictionary in dict_days_since_last:
    days_value_counts = calculate_days_since_last(df_concat, dictionary['input_col'], dictionary['output_col'], dictionary['threshold'])
    print(days_value_counts)


### Adressen: active_address_count

In [ ]:
# Create new column 'has_single_address', where 0 = 2 or more addresses, 1 = one address, 2 = less than 1 address 
def address_count(address):
    if address == 1:
        return 1
    elif address > 1:
        return 0
    else:
        return 2
    
df_concat['has_single_address'] = df_concat['active_address_count'].apply(address_count)
df_concat['has_single_address'].value_counts()

### Label: label

In [ ]:
df_concat['Label'].value_counts()

In [ ]:
df_concat.loc[
    ~(df_concat['is_fulltime_parent'].isna())
    &
    (df_concat['dataset'] == 'Prepilot')]

In [ ]:
df_concat

In [ ]:
# We have multiple ways of labeling, make uniform
dict_map_label = {1 : 'Onderzoekswaardig',
                    0 : 'Niet onderzoekswaardig'}

df_concat['Label'] = df_concat['Label'].replace(dict_map_label)
df_concat['Label'].value_counts()

### NaN to 'Onbekend'

In [ ]:
df_concat = df_concat.fillna('Onbekend')

## Final bias columns

### Export file

In [ ]:
df_concat.to_excel('20240203_bias_lhr_processed.xlsx', index=False)